# 05 — Prompt Templates

In LangChain, a **Prompt Template** is a predefined text structure that helps you build dynamic prompts. Instead of hardcoding entire prompts every time, you define **templates with variables** — this makes your code cleaner, reusable, and more adaptable.

## Setup

Make sure your `OPENAI_API_KEY` is available (e.g., via environment variables or `.env`).

In [ ]:
# ╔══════════════════════════════════════════════════════╗
# ║ Setup: Load environment variables & initialize model ║
# ╚══════════════════════════════════════════════════════╝

import os
from dotenv import load_dotenv, find_dotenv

# Load API keys from .env file
_ = load_dotenv(find_dotenv())

# Import your preferred model provider
from langchain_openai import ChatOpenAI
# from langchain_groq import ChatGroq

# Initialize model (OpenAI by default)
chat_model = ChatOpenAI(model="gpt-4o-mini")
# chat_model = ChatGroq(model="llama-3.1-70b-versatile")

print("✅ Environment loaded and model ready.")

## 🔹 Concept

A prompt template contains:
- **Fixed text** → content that always appears.
- **Variables** → placeholders filled with specific values at runtime.

**Example with variables:**

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """You are an expert in {topic}.
Answer the following question clearly: {question}"""

prompt = PromptTemplate(
    input_variables=["topic", "question"],
    template=template,
)

final_prompt = prompt.format(topic="history", question="Who was Julius Caesar?")
print(final_prompt)

You are an expert in history.
Answer the following question clearly: Who was Julius Caesar?


**Output:**
```
You are an expert in history.
Answer the following question clearly: Who was Julius Caesar?
```

## 🧠 ChatPromptTemplate

For chat models, LangChain provides **ChatPromptTemplate**, which lets you define prompts with roles (`system`, `human`, `ai`) and dynamic variables.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a {profession} expert in {topic}."),
        ("human", "Hello, {profession}, could you answer a question for me?"),
        ("ai", "Of course!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession="Scientist",
    topic="space exploration",
    user_input="What was the main goal of the Apollo 11 mission?",
)

response = chat_model.invoke(messages)
print(response.content)

The main goal of the Apollo 11 mission, which took place in July 1969, was to land humans on the Moon and safely return them to Earth. This mission aimed to fulfill President John F. Kennedy's 1961 goal of achieving a manned lunar landing before the end of the decade. Apollo 11 successfully accomplished this objective when astronauts Neil Armstrong and Buzz Aldrin landed on the Moon's surface on July 20, 1969, while Michael Collins remained in lunar orbit. Armstrong's famous words, "That's one small step for [a] man, one giant leap for mankind," marked a historic moment in human space exploration.


## 🧩 Basic Prompting Strategies

Different prompting strategies give different results. Some are more **creative**, others more **structured**.

```markdown
* Zero-Shot Prompt: "Classify the sentiment of this review: ..."
* Few-Shot Prompt: "Classify the sentiment of this review based on these examples: ..."
* Chain-of-Thought Prompt: "Classify the sentiment of this review based on these examples and explain the reasoning behind each answer."
```

## 🎯 Few-Shot Prompting

Few-shot prompting means you **show the model a few examples** of how you want it to respond, before asking your real question.

This helps guide its reasoning and output structure.

In [ ]:
# --- Deterministic English→Spanish translation (output-only) ---

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Strong, explicit instruction
system_msg = (
    "You are a professional English→Spanish translator. "
    "Translate the user's message into natural, idiomatic Spanish. "
    "Output ONLY the translation text. No explanations, no quotes, no preface."
)

# Few-shot examples to anchor behavior
examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
    {"input": "Good morning, everyone.", "output": "Buenos días a todos."},
    {"input": "How are you?", "output": "¿Cómo estás?"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}"), ("ai", "{output}")]
)

few_shot = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, examples=examples
)

# Final prompt with instruction + examples + user input
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_msg),
        few_shot,
        ("human", "{input}"),
    ]
)

parser = StrOutputParser()
chain = final_prompt | chat_model | parser

print(chain.invoke({"input": "How are you?"}))
# → expected: ¿Cómo estás?

¿Cómo estás?


## 🧠 Prompt Design Tips

✅ **Be explicit** — clearly tell the model its role and output format.

✅ **Use delimiters** — separate context, examples, or data to avoid confusion (e.g., triple backticks ``` or XML-style tags).

✅ **Prefer short, clear language** — models understand simpler prompts more reliably.

✅ **Always test variations** — small wording changes can drastically affect results.

✅ **Keep temperature low** when you need consistency or factual precision.

## ✅ Summary

- **Prompt templates** = reusable patterns with variables.
- **ChatPromptTemplate** = structured multi-message chat prompts.
- **FewShotPrompting** = guide the model with examples.
- Combine templates + structured prompts to achieve **reliable, consistent** outputs.

LangChain makes it easy to **compose** and **reuse** prompts — a key skill when building scalable LLM apps.